In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier, export_text
from sklearn.pipeline import Pipeline

data = pd.DataFrame({
    'Age': ['<21', '<21', '21-35', '>35', '>35', '>35', '21-35', '<21', '<21', '>35', '<21', '21-35', '21-35', '>35'],
    'Income': ['High', 'High', 'High', 'Medium', 'Low', 'Low', 'Low', 'Medium', 'Low', 'Medium', 'Medium', 'Medium', 'High', 'Medium'],
    'Gender': ['Male', 'Male', 'Male', 'Male', 'Female', 'Female', 'Female', 'Male', 'Female', 'Female', 'Female', 'Male', 'Female', 'Male'],
    'Marital Status': ['Single', 'Married', 'Single', 'Single', 'Single', 'Married', 'Married', 'Single', 'Married', 'Single', 'Married', 'Married', 'Single', 'Married'],
    'Buys': ['No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No']
})

X = data.drop(columns=['Buys'])
y = data['Buys']

preprocessor = ColumnTransformer(
    transformers=[
        ('age_income', OneHotEncoder(), ['Age', 'Income']),
        ('gender_marital', OneHotEncoder(), ['Gender', 'Marital Status'])
    ])

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', DecisionTreeClassifier(criterion="entropy", random_state=0))
])

pipeline.fit(X, y)

model = pipeline.named_steps['classifier']
tree_rules = export_text(model, feature_names=preprocessor.get_feature_names_out())
print(tree_rules)

test_sample = pd.DataFrame([{
    'Age': '<21',
    'Income': 'Low',
    'Gender': 'Female',
    'Marital Status': 'Married'
}])

prediction = pipeline.predict(test_sample)[0]
print("Prediction:", prediction)


|--- age_income__Age_21-35 <= 0.50
|   |--- gender_marital__Gender_Male <= 0.50
|   |   |--- gender_marital__Marital Status_Single <= 0.50
|   |   |   |--- age_income__Age_<21 <= 0.50
|   |   |   |   |--- class: No
|   |   |   |--- age_income__Age_<21 >  0.50
|   |   |   |   |--- class: Yes
|   |   |--- gender_marital__Marital Status_Single >  0.50
|   |   |   |--- class: Yes
|   |--- gender_marital__Gender_Male >  0.50
|   |   |--- age_income__Age_>35 <= 0.50
|   |   |   |--- class: No
|   |   |--- age_income__Age_>35 >  0.50
|   |   |   |--- gender_marital__Marital Status_Single <= 0.50
|   |   |   |   |--- class: No
|   |   |   |--- gender_marital__Marital Status_Single >  0.50
|   |   |   |   |--- class: Yes
|--- age_income__Age_21-35 >  0.50
|   |--- class: Yes

Prediction: Yes
